In [ ]:
# Loading the cv libraries 
import cv2
import os 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 

# Example pipeline for snow detection

In [ ]:
# Listing the images in the images dir 
images_dir = 'images'
images = os.listdir(images_dir)

# Saving the path to the first image 
image_path = os.path.join(images_dir, images[0])

# Ploting the image
image = cv2.imread(image_path)
plt.imshow(image)

In [ ]:
# We extract the center point from the image 
h_center, w_center = int(image.shape[0]/2), int(image.shape[1]/2)
print(w_center, h_center)

# Defining the buffer of pixels to crop around the center 
buffer = 100

# Calculating the top left and bottom right coordinates
top_left = (w_center - buffer, h_center - buffer)
bottom_right = (w_center + buffer, h_center + buffer)

# Cropping the image
cropped_image = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

# Ploting the extracted rectangle on the original image 
image_with_rect = image.copy()
cv2.rectangle(image_with_rect, top_left, bottom_right, (0,0,255), 10)
plt.imshow(image_with_rect)

In [ ]:
#Ploting the cropped image
plt.imshow(cropped_image)

In [ ]:
# Making the image gray
gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

# Calculating the mean of the gray image
gray_value = gray_image.mean()

# The closer the mean is to 255, the brighter the image is;
# The closer the mean is to 0, the darker the image is
print(gray_value)

# Ploting the gray image along with the mean gray value 
plt.imshow(gray_image, cmap='gray')
plt.title('Mean gray value: {}'.format(round(gray_value, 2)))

# Running all the images in the images dir

In [ ]:
# Creating the output dir 
cropped_image_dir = 'cropped_images'
if not os.path.exists(cropped_image_dir):
    os.makedirs(cropped_image_dir)

# Iterating over the images in the images dir and cropping them 
for image_name in tqdm(images):
    # Saving the path to the image 
    image_path = os.path.join(images_dir, image_name)
    
    # Loading the image
    image = cv2.imread(image_path)
    
    # Extracting the center point
    h_center, w_center = int(image.shape[0]/2), int(image.shape[1]/2)
    
    # Defining the buffer of pixels to crop around the center 
    buffer = 100
    
    # Calculating the top left and bottom right coordinates
    top_left = (w_center - buffer, h_center - buffer)
    bottom_right = (w_center + buffer, h_center + buffer)
    
    # Cropping the image
    cropped_image = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    
    # Calculating the mean of the cropped image
    gray_value = cropped_image.mean()

    # Saving the cropped image
    cropped_image_path = os.path.join(cropped_image_dir, image_name)

    # Creating a fig object with two axes: 
    # On the left there will be the original image with the rectangle
    # On the right there will be the cropped image with the mean gray value
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(image_name)

    # Ploting the original image with the rectangle
    axes[0].imshow(image)
    axes[0].set_title('Original image')
    axes[0].axis('off')
    axes[0].add_patch(plt.Rectangle(top_left, bottom_right[0] - top_left[0], bottom_right[1] - top_left[1], fill=False, color='red'))

    # Ploting the cropped image with the mean gray value
    axes[1].imshow(cropped_image, cmap='gray')
    axes[1].set_title('Mean gray value: {}'.format(round(gray_value, 2)))

    # Saving the ploted image
    plt.savefig(cropped_image_path)

    # Closing the fig object to free up memory
    plt.close(fig)